# 📐 Pi_AutoRunner — Linux Menu Support (Colab)

Dieses Notebook klont das Repo, kann (optional) die mitgelieferte `.deb` installieren
und führt die **Linux-Menu-Schritte** automatisch aus:

- `chmod +x *.sh` und `./start.sh`
- Falls nötig vorher: `python3-gmpy2` via `pi-install.sh`
- Separate Zellen für **Run from Source** und **Run from Installed .deb**


## 0) System vorbereiten

In [ ]:
!sudo apt-get update -y
!sudo apt-get install -y build-essential cmake libboost-dev nvidia-cuda-toolkit git dpkg-dev rsync
!python3 -V
!gcc --version | head -n1 || true
!nproc --all || true
!free -h || true
!df -h / || true
print("CUDA (nvcc) present?")
!nvcc --version || true


## 1) Repository klonen

In [ ]:
REPO_URL = "https://github.com/error-wtf/CALCULATION_OF_NUMBER_PI.git"
REPO_DIR = "/content/CALCULATION_OF_NUMBER_PI"

import os, shutil
if os.path.exists(REPO_DIR):
    shutil.rmtree(REPO_DIR)

!git clone --depth=1 "$REPO_URL" "$REPO_DIR"
!find "$REPO_DIR" -maxdepth 2 -type f -printf "%p\n" | sort


## 2) (Optional) .deb installieren (falls gewünscht)

In [ ]:
import os, glob
DEB_NAME = "pi-error-wtf-v1.0-Linux.deb"
debs = [p for p in glob.glob(os.path.join(REPO_DIR, "**", "*.deb"), recursive=True)
         if os.path.basename(p) == DEB_NAME]
if debs:
    deb_path = debs[0]
    print("Gefunden:", deb_path)
    !chmod a+x "$deb_path"
    !sudo dpkg -i "$deb_path" || true
    print("\nFalls Abhängigkeiten fehlen:  sudo apt-get -f install -y")
else:
    print(f"{DEB_NAME} nicht gefunden – Schritt übersprungen.")


## 3) Linux Menu — Dependencies installieren (falls nötig)

In [ ]:
# Variante A: Direkt via apt (schnell)
!sudo apt-get install -y python3-gmpy2 || true

# Variante B: Falls das Projekt ein eigenes Install-Skript hat:
%cd /content/CALCULATION_OF_NUMBER_PI
!sudo chmod a+x *.sh || true
import os
if os.path.exists("pi-install.sh"):
    !sudo ./pi-install.sh || true
else:
    print("pi-install.sh nicht gefunden – benutze apt-Install oben.")


## 4) Linux Menu — Run (`chmod +x *.sh` und `./start.sh`)

In [ ]:
%cd /content/CALCULATION_OF_NUMBER_PI
!sudo chmod a+x *.sh || true
import os
if os.path.exists("start.sh"):
    print("Starte ./start.sh ...")
    !sudo ./start.sh
else:
    print("start.sh nicht gefunden. Bitte prüfe den Repo-Inhalt.")


---
## ▶️ Separat: Run from Source (optional)
Erkennt den Entry-Point automatisch und startet direkt aus dem Repo.

In [ ]:
import subprocess, sys, shlex, glob, os
os.chdir(REPO_DIR)

candidates = ["main.py","runner.py","run.py"]
for p in sorted(glob.glob("*.py"), key=lambda p: os.path.getsize(p), reverse=True):
    if p not in candidates:
        candidates.append(p)

ep = ""
for c in candidates:
    if os.path.exists(c):
        ep = c
        break

print("Detected ENTRYPOINT:", ep or "<none>")
if ep:
    cmd = ["python3","-u",ep]
    print("Running:", " ".join(shlex.quote(x) for x in cmd))
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    for line in p.stdout:
        sys.stdout.write(line)
    p.wait()
    print("\n-- exit code:", p.returncode)
else:
    print("Kein Entry-Point gefunden.")


## ▶️ Separat: Run from Installed .deb (optional)
Versucht `pi-error-wtf --help`, dann `pi-calc`, dann `/usr/local/bin/pi*`, danach Fallback auf `/opt/CALCULATION_OF_NUMBER_PI/*.py`.

In [ ]:
import subprocess, sys, shlex, glob, os

def try_run(cmd):
    try:
        print("Trying:", " ".join(shlex.quote(c) for c in cmd))
        p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        for line in p.stdout:
            sys.stdout.write(line)
        p.wait()
        print("\n-- exit code:", p.returncode)
        return p.returncode == 0
    except FileNotFoundError:
        return False

if not try_run(["pi-error-wtf","--help"]):
    if not try_run(["pi-calc","--help"]):
        bin_dir = "/usr/local/bin"
        found = False
        if os.path.isdir(bin_dir):
            for f in sorted(os.listdir(bin_dir)):
                p = os.path.join(bin_dir, f)
                try:
                    if os.path.isfile(p) and os.access(p, os.X_OK) and f.startswith("pi"):
                        if try_run([p, "--help"]):
                            found = True
                            break
                except Exception:
                    pass
        if not found:
            opt = "/opt/CALCULATION_OF_NUMBER_PI"
            if os.path.isdir(opt):
                py = ["main.py","runner.py","run.py"]
                for pth in sorted(glob.glob(os.path.join(opt,"*.py")), key=lambda q: os.path.getsize(q), reverse=True):
                    b = os.path.basename(pth)
                    if b not in py:
                        py.append(b)
                ep = ""
                for c in py:
                    if os.path.exists(os.path.join(opt,c)):
                        ep = os.path.join(opt,c); break
                if ep:
                    try_run(["python3","-u",ep,"--help"])
                else:
                    print("Kein Python-Entry unter /opt/CALCULATION_OF_NUMBER_PI gefunden.")
            else:
                print("/opt/CALCULATION_OF_NUMBER_PI nicht vorhanden – ist die .deb installiert?")
